**source:**
* https://thedatafrog.com/en/articles/docker-influxdb-grafana/

**resources:**
* https://docs.influxdata.com/influxdb/v1.8/introduction/get-started/
* https://www.influxdata.com/blog/getting-started-python-influxdb/
* https://influxdb-python.readthedocs.io/en/latest/

In [2]:
# instaling influxdb library
!pip install influxdb

     |████████████████████████████████| 77 kB 350 kB/s eta 0:00:011
     |████████████████████████████████| 118 kB 1.3 MB/s eta 0:00:01
  Created wheel for msgpack: filename=msgpack-0.6.1-cp38-cp38-linux_x86_64.whl size=343732 sha256=b4ac7b1bffd92aac179f248b95a8c1662987dee3281c432db616bc05893a0d3d
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/2e/45/9ae160fc31c10e4b799d0ebc32ba82b32f1b057e0ebf28ea82
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.0
    Uninstalling msgpack-1.0.0:
      Successfully uninstalled msgpack-1.0.0


In [1]:
from influxdb import InfluxDBClient
from influxdb import exceptions

In [2]:
try:
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    print('Connected to InfluxDB')
except Exception as e:
    print('Connection error: {}'.format(str(e)))

Connected to InfluxDB


In [4]:
# Creating a test db

client.create_database('testdb')

In [3]:
# Let's check the list of databases

client.get_list_database()

[{'name': '_internal'}, {'name': 'testdb'}]

To send data to influxdb we have to use a JSON format to indicate 

1.- Measurement (mandatory field)
2.- Tags (optional field)
3.- Time (optional field. If no time value is given Influx db will use the system's timestamp)
4.- Field (mandatory)

In [8]:
# from influx db python documentation
json_body = [
    {
        "measurement": "cpu_load_short",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2009-11-10T23:00:00Z",
        "fields": {
            "value": 0.64
        }
    }
]


In [10]:
# write_points function also can take de different fields as parameters. See documentation.

client.write_points(json_body, database='testdb')

True

In [17]:
# Let's check the value was stored correctly

result = client.query('select "host", "region", "value" from cpu_load_short;', database='testdb')

In [19]:
# seems like is working just right
print("Result: {0}".format(result))

Result: ResultSet({'('cpu_load_short', None)': [{'time': '2009-11-10T23:00:00Z', 'host': 'server01', 'region': 'us-west', 'value': 0.64}]})


* The **measurement** acts as a container for tags, fields, and the time column, and the measurement name is the description of the data that are stored in the associated fields. 
* **Field** values are your data; they can be strings, floats, integers, or Booleans, and, because InfluxDB is a time series database, a field value is always associated with a timestamp
* **Tags** are made up of tag keys and tag values. Both tag keys and tag values are stored as strings and record metadata.Tags are optional. You don’t need to have tags in your data structure, but it’s generally a good idea to make use of them because, unlike fields, tags are indexed.
* A **retention policy** describes how long InfluxDB keeps data (DURATION) and how many copies of this data is stored in the cluster (REPLICATION).
* A **series** is a collection of points that share a measurement, tag set, and field key.
* A **point** represents a single data record that has four components: a measurement, tag set, field set, and a timestamp.
* **Continuous queries (CQ)** are InfluxQL queries that run automatically and periodically on realtime data and store query results in a specified measurement

In [9]:
client.switch_database('testdb')

In [10]:
client.get_list_measurements()

[{'name': 'cpu_load_short'}, {'name': 'mxx'}]

In [11]:
client.get_list_series()

['cpu_load_short,host=server01,region=us-west',
 'mxx,measurement_type=day,source=yahoo']

In [14]:
result = client.query('select "measurement_type", "source", "value" from mxx;', database='testdb')

In [16]:
print(result)

ResultSet({'('mxx', None)': [{'time': '2020-07-17T18:00:13.389995Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-17T18:35:43.009230Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-17T23:40:32.066297Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-17T23:52:27.720545Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-17T23:52:29.563945Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-18T00:02:41.960505Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-18T00:02:43.653096Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-18T00:02:45.232742Z', 'measurement_type': 'day', 'source': 'yahoo', 'value': 36327.83984375}, {'time': '2020-07-18T00:02:46.973521Z', 'measurement_type': 'day', 'source'